In [4]:
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
import rasterio
import matplotlib.pyplot as plt
import os

fp = os.path.join('/', 
             'Users', 
             'jpcohen', 
             'EDS-220', 
             'eds220-hwk4', 
             'data', 
             'landsat8-2018-01-26-sb-simplified.nc')

In [10]:
landsat = rioxr.open_rasterio(fp)
landsat

<xarray.Dataset> Size: 25MB
Dimensions:      (band: 1, x: 870, y: 731)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 7kB 1.213e+05 1.216e+05 ... 3.557e+05 3.559e+05
  * y            (y) float64 6kB 3.952e+06 3.952e+06 ... 3.756e+06 3.755e+06
    spatial_ref  int64 8B 0
Data variables:
    red          (band, y, x) float64 5MB ...
    green        (band, y, x) float64 5MB ...
    blue         (band, y, x) float64 5MB ...
    nir08        (band, y, x) float64 5MB ...
    swir22       (band, y, x) float64 5MB ...

In [20]:
landsat.isel(band = 0, x = 5, y = 7)

<xarray.Dataset> Size: 72B
Dimensions:      ()
Coordinates:
    band         int64 8B 1
    x            float64 8B 1.227e+05
    y            float64 8B 3.951e+06
    spatial_ref  int64 8B 0
Data variables:
    red          float64 8B ...
    green        float64 8B ...
    blue         float64 8B ...
    nir08        float64 8B ...
    swir22       float64 8B ...

In [9]:
print(landsat.dims, '\n\n', 
      landsat.attrs, '\n\n', 
      landsat.coords, '\n\n')

FrozenMappingWarningOnValuesAccess({'band': 1, 'x': 870, 'y': 731}) 

 {} 

 Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 7kB 1.213e+05 1.216e+05 ... 3.557e+05 3.559e+05
  * y            (y) float64 6kB 3.952e+06 3.952e+06 ... 3.756e+06 3.755e+06
    spatial_ref  int64 8B 0 




# Preliminary Summary of the Landsat Data

The data has 3 dimensions. The first, appearing to be a dummy dimension to refer to a single band of data. Rather than multiple bands being layered on top of each other like in a traditional multi-band raster, each cell contains every respective value for each variable. 

The other two dimensions are x and y, refering to longtitude and latitude. They have a unit of a meter, and a crs of EPSG:32611.

Furthermore, the data has 5 variables: red, green, blue, nir08, and swir22, referring to red, green, blue, near infrared, and short wave infrared respectively. 

The data otherwise has no attributes.